In [1]:
using Pkg
Pkg.precompile()
Pkg.activate("..")
Pkg.instantiate()

  Activating project at `~/awid/automaticDifferentiation/ADLib`


In [2]:
using JLD2
X_train = load("./imdb_dataset_prepared.jld2", "X_train")
y_train = load("./imdb_dataset_prepared.jld2", "y_train")
X_test = load("./imdb_dataset_prepared.jld2", "X_test")
y_test = load("./imdb_dataset_prepared.jld2", "y_test")
nothing
using ADLib

batch_size=64
dataset = ADLib.DataLoader((X_train, y_train), batchsize=batch_size, shuffle=true)

ADLib.DataLoader{Tuple{LinearAlgebra.Adjoint{Float32, Matrix{Float32}}, BitMatrix}}((Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Bool[1 1 … 0 0]), 64, true, [[934, 4107, 3341, 7747, 411, 510, 6033, 3430, 3043, 1638  …  1793, 7315, 4230, 6973, 3265, 2132, 533, 3493, 1276, 3606], [2905, 40, 6860, 2297, 1326, 4580, 5633, 2521, 5752, 5608  …  1294, 7973, 5331, 876, 3305, 5780, 3258, 7565, 6957, 2047], [6255, 6040, 3961, 971, 3220, 6385, 792, 857, 1410, 6921  …  211, 7435, 5280, 3686, 3893, 82, 7919, 3264, 6903, 4411], [5059, 6285, 6000, 34, 7230, 7134, 7768, 7634, 7547, 111  …  772, 4009, 6242, 3347, 2743, 3610, 1700, 1369, 3377, 239], [3671, 3199, 3107, 4209, 4240, 3443, 7555, 3419, 5955, 5689  …  6116, 5975, 1750, 5169, 3019, 6120, 3057, 4592, 5916, 1764], [215, 3163, 2296, 7511, 6684, 3251, 2165, 4479, 3221, 6156  …  1667, 2526, 1533, 6449, 1690, 4604, 568, 5007, 2040, 416], [3704, 4566, 1205, 3170, 5602, 4681, 2896, 6577, 7047, 2579  …  4505

In [3]:
using Printf

model = ADLib.Sequence(
    ADLib.Dense{Float64}(ADLib.ReLU(), size(X_train, 1), 32),
    ADLib.Dense{Float64}(ADLib.Sigmoid(), 32, 1)
)
epochs = 5
for epoch in 1:epochs
    t_loss = 0.0
    t_accuracy = 0.0
    gradient = zeros(Float64, 1, 64)
    num_samples = 0
    opt = ADLib.Adam()
    t = @elapsed begin
        for (x, y) in dataset
            output = model(x)
            t_loss += ADLib.binary_crossentropy(y, output)
            t_accuracy += ADLib.binary_accuracy(y, output)
            #println(size(y))
            gradient = ADLib.binary_crossentropy_gradient(y, output) ./ batch_size
            ADLib.backward_pass(model, gradient)
            num_samples += 1
            ADLib.update_weights(model, opt)
            #println("next loop")
        end   
        
        train_loss = t_loss / num_samples
        train_acc = t_accuracy / num_samples

        y = model(X_test)
        test_loss = ADLib.binary_crossentropy(y_test, y)
        test_acc = ADLib.binary_accuracy(y_test, y)
    end
            
    println(@sprintf("Epoch: %d (%.2fs) \tTrain: (l: %.2f, a: %.2f) \tTest: (l: %.2f, a: %.2f)", 
        epoch, t, train_loss, train_acc, test_loss, test_acc))
end

Epoch: 1 (7.23s) 	Train: (l: 0.52, a: 0.76) 	Test: (l: 0.39, a: 0.84)
Epoch: 2 (3.18s) 	Train: (l: 0.09, a: 0.97) 	Test: (l: 0.52, a: 0.82)
Epoch: 3 (3.21s) 	Train: (l: 0.05, a: 0.98) 	Test: (l: 0.75, a: 0.81)
Epoch: 4 (3.58s) 	Train: (l: 0.02, a: 0.99) 	Test: (l: 1.05, a: 0.80)
Epoch: 5 (3.17s) 	Train: (l: 0.03, a: 0.99) 	Test: (l: 0.93, a: 0.83)


In [4]:
using Flux, Printf, Statistics

#dataset = Flux.DataLoader((X_train, y_train), batchsize=64, shuffle=true)

model = Chain(
    Dense(size(X_train, 1), 32, relu),
    Dense(32, 1, sigmoid)
)

loss(m, x, y) = Flux.Losses.binarycrossentropy(m(x), y)
accuracy(m, x, y) = mean((m(x) .> 0.5) .== (y .> 0.5))

opt = Flux.setup(Adam(), model)
epochs = 5
for epoch in 1:epochs
    total_loss = 0.0
    total_acc = 0.0
    num_samples = 0

    t = @elapsed begin
        for (x, y) in dataset
            grads = Flux.gradient(model) do m
                l = loss(m, x, y)
                total_loss += l
                total_acc += accuracy(m, x, y)
                return l
            end
            Optimisers.update!(opt, model, grads[1])
            num_samples += 1
        end

        train_loss = total_loss / num_samples
        train_acc = total_acc / num_samples

        test_acc = accuracy(model, X_test, y_test)
        test_loss = loss(model, X_test, y_test)
    end

    println(@sprintf("Epoch: %d (%.2fs) \tTrain: (l: %.2f, a: %.2f) \tTest: (l: %.2f, a: %.2f)", 
        epoch, t, train_loss, train_acc, test_loss, test_acc))
end

Epoch: 1 (15.55s) 	Train: (l: 0.64, a: 0.79) 	Test: (l: 0.58, a: 0.85)
Epoch: 2 (2.33s) 	Train: (l: 0.45, a: 0.91) 	Test: (l: 0.44, a: 0.86)
Epoch: 3 (2.14s) 	Train: (l: 0.29, a: 0.94) 	Test: (l: 0.36, a: 0.87)
Epoch: 4 (2.37s) 	Train: (l: 0.20, a: 0.96) 	Test: (l: 0.33, a: 0.87)
Epoch: 5 (2.30s) 	Train: (l: 0.14, a: 0.98) 	Test: (l: 0.32, a: 0.87)
